# Fine-tuning

We create two fine-tuning jobs on the Together AI platform, one for each dataset. Here, we fine-tune on Qwen2.5-32B-Instruct.

In [1]:
from dotenv import load_dotenv; load_dotenv()
import random as random
import openai
import os 
import time
from pathlib import Path
from together import Together


api_key = os.getenv("VALUESLAB_OPENAI")
client = openai.OpenAI(api_key=api_key)
fireworks_api_key = os.getenv("FIREWORKS_API_KEY")
fireworks_account_id = os.getenv("FIREWORKS_ACCOUNT_ID")
together_api_key = os.getenv("TOGETHER_API_KEY") 

In [ ]:
# ============== CONFIGURATION ==============
EPOCHS = 5
LEARNING_RATE = 2e-4
LORA_RANK = 32
BASE_MODEL = "Qwen/Qwen2.5-32B-Instruct"

together_client = Together(api_key=os.getenv("TOGETHER_API_KEY"))

variants = {
    "experimental": Path("data/experimental_data.jsonl"),
    "control": Path("data/control_data.jsonl")
}

# ============== UPLOAD & FINE-TUNE ==============
created = {}

for name, path in variants.items():
    ts = time.strftime("%Y%m%d-%H%M%S")
    print(f"\n=== {name.upper()} ===")
    
    # 1) Upload file
    print(f"Uploading {path}...")
    file_resp = together_client.files.upload(file=path)
    file_id = file_resp.id
    print(f"File uploaded: {file_id}")
    
    # 2) Wait for file processing
    print("Waiting for file processing...")
    while True:
        file_status = together_client.files.retrieve(file_id)
        print(f"  processed: {file_status.processed}")
        if file_status.processed:
            break
        time.sleep(5)
    
    # 3) Create fine-tuning job
    print("Starting fine-tune job...")
    job = together_client.fine_tuning.create(
        training_file=file_id,
        model=BASE_MODEL,
        n_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        lora_r=LORA_RANK,
        lora=True,
        suffix=f"{name}-{ts}"
    )
    print(f"Job created: {job.id}")
    
    created[name] = {
        "job_id": job.id,
        "file_id": file_id,
        "timestamp": ts
    }

print("\n" + "="*50)
print("Jobs launched!")
for name, info in created.items():
    print(f"  {name}: {info['job_id']}")
print("\nCheck status at: https://api.together.xyz/jobs")


=== EXPERIMENTAL ===
Uploading data/experimental_data.jsonl...


Validating file: 10000 lines [00:00, 199020.82 lines/s]
Uploading file experimental_data.jsonl: 100%|██████████| 6.24M/6.24M [00:00<00:00, 8.49MB/s]


File uploaded: file-d7f54191-3dc5-46c6-be77-62e4c971c8ae
Waiting for file processing...


message="train_on_inputs is not set for SFT training, it will be set to 'auto'"
message="train_on_inputs is not set for SFT training, it will be set to 'auto'"


  processed: True
Starting fine-tune job...
Job created: ft-35fc6dba-689d

=== CONTROL ===
Uploading data/control_data.jsonl...


Validating file: 10000 lines [00:00, 171797.72 lines/s]
Uploading file control_data.jsonl: 100%|██████████| 6.66M/6.66M [00:00<00:00, 15.3MB/s]


File uploaded: file-91cdf481-a2e9-47fd-a7f6-2ebef4431a91
Waiting for file processing...
  processed: True
Starting fine-tune job...
Job created: ft-3cd5265d-f8bd

Jobs launched!
  experimental: ft-35fc6dba-689d
  control: ft-3cd5265d-f8bd

Check status at: https://api.together.xyz/jobs
